In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device
import wandb
import datetime
import pytz

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [6]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('../../../baseline/mmdetection/configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')

root='../dataset/'

# dataset config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
cfg.data.train.ann_file = root + 'train_area_10000.json' # train json 정보
cfg.data.train.pipeline[2]['img_scale'] = (512,512) # Resize

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.test.pipeline[1]['img_scale'] = (512,512) # Resize

cfg.data.samples_per_gpu = 4

# wandb setting
kst = pytz.timezone('Asia/Seoul')
now = datetime.datetime.now(kst)
date = now.strftime('%Y_%m_%d_%H_%M_%S_')
project_name = "Mini"
exp_name = date + "FasterRCNN_Area10000"
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='MMDetWandbHook',
         init_kwargs={'project': project_name, 'name' : exp_name},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=0,
         bbox_score_thr=0.3)]

cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/' + exp_name

cfg.model.roi_head.bbox_head.num_classes = 10

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
cfg.device = get_device()

In [7]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


In [8]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 4871, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 2453  | 1 [Paper]     | 4719  | 2 [Paper pack]  | 664   | 3 [Metal]   | 680   | 4 [Glass]    | 802   |
| 5 [Plastic]       | 2071  | 6 [Styrofoam] | 1073  | 7 [Plastic bag] | 4312  | 8 [Battery] | 140   | 9 [Clothing] | 441   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [7]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

2023-05-09 12:01:21,856 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2023-05-09 12:01:21,857 - mmcv - INFO - load model from: torchvision://resnet50
2023-05-09 12:01:21,858 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2023-05-09 12:01:22,080 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2023-05-09 12:01:22,108 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2023-05-09 12:01:22,135 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2023-05-09 12:01:22,144 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

In [8]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=False)

2023-05-09 12:01:28,641 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2023-05-09 12:01:28,646 - mmdet - INFO - Start running, host: root@900ed28a7422, work_dir: /opt/ml/baseline/mmdetection/work_dirs/2023_05_09_21_00_28_FasterRCNN_Area10000
2023-05-09 12:01:28,647 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MMDetWandbHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook      

2023-05-09 12:01:33,807 - mmdet - WARNING - No meta information found in the runner. 
2023-05-09 12:01:33,809 - mmdet - WARNING - To log evaluation or checkpoint metadata in MMDetWandbHook, `EvalHook` or `DistEvalHook` in mmdet is required, please check whether the validation is enabled.
2023-05-09 12:01:48,685 - mmdet - INFO - Epoch [1][50/1218]	lr: 1.978e-03, eta: 1:12:12, time: 0.297, data_time: 0.052, memory: 2103, loss_rpn_cls: 0.3772, loss_rpn_bbox: 0.0367, loss_cls: 0.6308, acc: 88.4395, loss_bbox: 0.1223, loss: 1.1670, grad_norm: 7.0349
2023-05-09 12:02:00,817 - mmdet - INFO - Epoch [1][100/1218]	lr: 3.976e-03, eta: 1:05:19, time: 0.243, data_time: 0.007, memory: 2103, loss_rpn_cls: 0.0950, loss_rpn_bbox: 0.0329, loss_cls: 0.2882, acc: 94.6396, loss_bbox: 0.2094, loss: 0.6254, grad_norm: 2.1328
2023-05-09 12:02:12,619 - mmdet - INFO - Epoch [1][150/1218]	lr: 5.974e-03, eta: 1:02:22, time: 0.236, data_time: 0.006, memory: 2103, loss_rpn_cls: 0.0719, loss_rpn_bbox: 0.0251, loss_c

learning_rate,▃██████████████████████████▂▂▂▂▂▂▂▂▂▂▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▄▃▁▃▃▂▂▄▁▅▃▃▄▂▃▄▃▆▄▃▅▄▆▆▅▃▅▆▆▅█▇▇▇▅▆▇▇▆▆
train/grad_norm,▅█▄▅▄▄▄▂▃▃▃▄▄▆▃▄▅▂▅▅▄▄▃▃▃▆▄▂▃▄▁▃▃▃▄▇▃▂▄▄
train/loss,▇▇█▆▆▆▆▄▆▄▅▅▅▆▅▅▅▃▄▅▃▅▃▂▃▅▃▃▃▃▁▂▂▂▃▃▂▂▂▃
train/loss_bbox,▇▆█▅▅▆▅▄▆▃▄▅▄▆▅▅▅▂▄▄▄▄▃▂▃▅▃▄▄▄▁▂▃▂▄▄▃▂▃▃
train/loss_cls,▇██▇▆▇▇▅▇▄▅▆▅▆▅▄▅▃▄▅▃▅▃▃▃▅▃▃▃▃▁▂▂▂▃▃▂▁▂▃
train/loss_rpn_bbox,▇▇█▆▇▅▇▅▅▆▄▄▄▇▅▅▅▃▄▄▄▇▄▃▃▆▃▄▄▄▁▃▃▂▅▄▃▃▂▄
train/loss_rpn_cls,█▇▄▅▄▄▄▃▃▄▃▃▃▅▃▃▃▂▃▃▂▃▂▂▂▂▂▂▁▂▁▁▁▂▁▂▁▁▁▂
learning_rate,0.0002
momentum,0.9
